# 06b: Calculate costs

In this script, we calculate the following cost estimates for regional averages of impacts

- low estimate (0.2 quantile)
- median cost
- damage costs
- prevention costs
- taxation costs
- budget constraint

In [1]:
%run common_definitions.py

In [2]:
import pandas as pd
import numpy as np
import xarray as xr

In [3]:
output_fp = "../output/" + BW_PROJECTNAME

In [4]:
mfs_perspectives_cc = xr.open_dataarray("../data/mfs_perspectives_extended_euro_{}.nc".format(EURO_REF_YEAR))
mfs_mc_sample_cc = xr.open_dataarray("../data/mfs_monte_carlo_sample_extended_euro{}.nc".format(str(EURO_REF_YEAR)))

We calculate costs for the regional averages.

In [5]:
def get_quantile_costs(impact_df, q, mfs, contract_dim):
    all_costs = xr.DataArray(
        np.tensordot(impact_df[list(mfs.coords["LCIA method"].values)].to_numpy(),
                    mfs.to_numpy(),
                    axes=([1,], [contract_dim,])),
        coords={
            "tech index": list(range(len(impact_df))),
            "impact category": list(mfs.coords["impact category"].values),
            "sample index": list(range(mfs.shape[1]))
        }
    )

    a = all_costs.to_numpy()
    qcosts = np.where(np.all(a < 0, axis=2),
                      np.quantile(a, 1-q, axis=2),
                      np.quantile(a, q, axis=2))

    return pd.DataFrame(qcosts,
                        index=impact_df.index,
                        columns=list(mfs.coords["impact category"].values))

def get_perspectives_cost(impact_df, mfs):
    df_mfs = mfs_perspectives_cc.to_dataframe().reset_index().pivot(
        index=["perspective", "impact category"],
        columns=["LCIA method"],
        values="value")
    
    return pd.DataFrame(
        np.tensordot(all_averages[df_mfs.columns].to_numpy(), df_mfs.to_numpy().T, axes=1),
        index=all_averages.index,
        columns=df_mfs.index
    )


### Full impacts

In [6]:
all_averages = pd.read_csv(output_fp+"/impacts_regional_averages.csv").set_index(
    ["short name", "sector", "scenario", "year", "regional average"])

low_costs = get_quantile_costs(all_averages, 0.2, mfs_mc_sample_cc, 2)
median_costs = get_quantile_costs(all_averages, 0.5, mfs_mc_sample_cc, 2)

low_costs.reset_index(inplace=True)
median_costs.reset_index(inplace=True)

low_costs["perspective"] = "low estimate"
median_costs["perspective"] = "median"

perspectives_costs = get_perspectives_cost(all_averages, mfs_perspectives_cc)

perspectives_costs = perspectives_costs.melt(ignore_index=False).reset_index().pivot(
    columns="impact category", values="value",
    index=["short name", "sector", "scenario", "year", "regional average", "perspective"])

costs_all_perspectives = pd.concat(
    (
        low_costs,
        median_costs,
        perspectives_costs.reset_index()
    ),
    axis=0
).set_index(["short name", "sector", "scenario", "year", "regional average", "perspective"]).sort_index()

costs_all_perspectives.to_csv(output_fp+"/costs_regional_averages.csv")

### Direct impacts

In [6]:
all_averages = pd.read_csv(output_fp+"/direct_impacts_regional_averages.csv").set_index(
    ["short name", "sector", "scenario", "year", "regional average"])

low_costs = get_quantile_costs(all_averages, 0.2, mfs_mc_sample_cc, 2)
median_costs = get_quantile_costs(all_averages, 0.5, mfs_mc_sample_cc, 2)

low_costs.reset_index(inplace=True)
median_costs.reset_index(inplace=True)

low_costs["perspective"] = "low estimate"
median_costs["perspective"] = "median"

perspectives_costs = get_perspectives_cost(all_averages, mfs_perspectives_cc)

perspectives_costs = perspectives_costs.melt(ignore_index=False).reset_index().pivot(
    columns="impact category", values="value",
    index=["short name", "sector", "scenario", "year", "regional average", "perspective"])

costs_all_perspectives = pd.concat(
    (
        low_costs,
        median_costs,
        perspectives_costs.reset_index()
    ),
    axis=0
).set_index(["short name", "sector", "scenario", "year", "regional average", "perspective"]).sort_index()

costs_all_perspectives.to_csv(output_fp+"/direct_costs_regional_averages.csv")

### Indirect impacts

In [14]:
all_averages = pd.read_csv(output_fp+"/indirect_impacts_regional_averages.csv").set_index(
    ["short name", "sector", "scenario", "year", "regional average"])

low_costs = get_quantile_costs(all_averages, 0.2, mfs_mc_sample_cc, 2)
median_costs = get_quantile_costs(all_averages, 0.5, mfs_mc_sample_cc, 2)

low_costs.reset_index(inplace=True)
median_costs.reset_index(inplace=True)

low_costs["perspective"] = "low estimate"
median_costs["perspective"] = "median"

perspectives_costs = get_perspectives_cost(all_averages, mfs_perspectives_cc)

perspectives_costs = perspectives_costs.melt(ignore_index=False).reset_index().pivot(
    columns="impact category", values="value",
    index=["short name", "sector", "scenario", "year", "regional average", "perspective"])

costs_all_perspectives = pd.concat(
    (
        low_costs,
        median_costs,
        perspectives_costs.reset_index()
    ),
    axis=0
).set_index(["short name", "sector", "scenario", "year", "regional average", "perspective"]).sort_index()

costs_all_perspectives.to_csv(output_fp+"/indirect_costs_regional_averages.csv")